In [40]:
# %pip install opencv-python numpy 


In [41]:
import json
import glob
import re


train_path = './data-training-pisang/*'  # Gunakan * untuk mencocokkan semua file

train_data_label = []

# Iterasi melalui file dalam folder
for folder_path in glob.glob(train_path):
    train_file_path = folder_path + '/*'

    label = ''

    # Pola regex untuk mencari bagian setelah tanda "/" terakhir
    pattern = r'/([^/]+)$'

    # Menggunakan re.search untuk mencocokkan pola
    match = re.search(pattern, folder_path)

    # Jika cocok, Anda dapat mengambil bagian yang cocok dengan grup (group)
    if match:
        label = match.group(1)
    else:
        print("Tidak cocok")

    for file_path in glob.glob(train_file_path):
        train_data_label.append([file_path, label])

# simpan lokasi file ke json
with open('data/data_train_locations.json', 'w') as outfile:
    outfile.write(json.dumps(train_data_label, sort_keys=True, indent=4))

In [51]:
import cv2
import numpy as np
import pandas as pd
import os


def proses_gambar(img_path):
    # Baca gambar
    image = cv2.imread(img_path)

    # Dapatkan dimensi gambar (lebar dan tinggi)
    height, width, _ = image.shape

    # Tentukan ukuran gambar yang dipotong
    new_width = 500  # Misalnya, 200 piksel
    new_height = 500  # Misalnya, 200 piksel

    # Hitung koordinat x dan y untuk memotong gambar di tengah
    x = (width - new_width) // 2
    y = (height - new_height) // 2

    # Potong gambar
    cropped_image = image[y:y+new_height, x:x+new_width]

    # Ekstraksi komponen warna RGB
    blue_channel = cropped_image[:, :, 0]
    green_channel = cropped_image[:, :, 1]
    red_channel = cropped_image[:, :, 2]

    # Menghitung rata-rata komponen warna
    average_blue = np.mean(blue_channel) / 255
    average_green = np.mean(green_channel) / 255
    average_red = np.mean(red_channel) / 255
    
    return [average_red, average_green, average_blue]

with open('./data/data_train_locations.json') as locations_json:
    locations_with_label = json.load(locations_json)

np_locations = np.array(locations_with_label)

X_train = np.array([proses_gambar(image_path) for image_path in np_locations[:, 0]])
y_train = np_locations[:, 1]

In [59]:
from sklearn.neighbors import KNeighborsClassifier

# init KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)


data_test = np.array([
  './data/FOTO MENTAH/2023_06_21_12_25_IMG_5139.JPG',
])

# Memisahkan fitur (X) dan label (y) untuk data pengujian
X_test = np.array([proses_gambar(image_path) for image_path in data_test])

y_pred = knn.predict(X_test)

print(y_pred[0])

mentah


In [50]:
with open('./data/data_train_locations.json') as locations_json:
    locations_with_label = json.load(locations_json)

for i, item in enumerate(locations_with_label):
    image = cv2.imread(item[0])

    # Dapatkan dimensi gambar (lebar dan tinggi)
    height, width, _ = image.shape

    # Tentukan ukuran gambar yang dipotong
    new_width = 500  
    new_height = 500  

    # Hitung koordinat x dan y untuk memotong gambar di tengah
    x = (width - new_width) // 2
    y = (height - new_height) // 2

    # Potong gambar
    cropped_image = image[y:y+new_height, x:x+new_width]

    cv2.imwrite(f'./data/image/{i}.jpg', cropped_image)